In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, avg, count, min, max, row_number

data_transactions = [
    (1, "2011-01-01", 500), (1, "2011-01-15", 50), (1, "2011-01-22", 250),
    (1, "2011-01-24", 75), (1, "2011-01-26", 125), (1, "2011-01-28", 175),
    (2, "2011-01-01", 500), (2, "2011-01-15", 50), (2, "2011-01-22", 25),
    (2, "2011-01-23", 125), (2, "2011-01-26", 200), (2, "2011-01-29", 250),
    (3, "2011-01-01", 500), (3, "2011-01-15", 50), (3, "2011-01-22", 5000),
    (3, "2011-01-25", 550), (3, "2011-01-27", 95), (3, "2011-01-30", 2500)
]

columns_transactions = ["AccountId", "TranDate", "TranAmt"]
df_transactions = spark.createDataFrame(data_transactions, columns_transactions)

data_logical = [
    (1, "George", 800), (2, "Sam", 950), (3, "Diane", 1100),
    (4, "Nicholas", 1250), (5, "Samuel", 1250), (6, "Patricia", 1300),
    (7, "Brian", 1500), (8, "Thomas", 1600), (9, "Fran", 2450),
    (10, "Debbie", 2850), (11, "Mark", 2975), (12, "James", 3000),
    (13, "Cynthia", 3000), (14, "Christopher", 5000)
]

columns_logical = ["RowID", "FName", "Salary"]
df_logical = spark.createDataFrame(data_logical, columns_logical)

In [0]:
window_spec = Window.partitionBy("AccountId").orderBy("TranDate")

df_transactions = df_transactions.withColumn("RunTotalAmt", sum("TranAmt").over(window_spec))
df_transactions = df_transactions.withColumn("RunAvg", avg("TranAmt").over(window_spec))
df_transactions = df_transactions.withColumn("RunTranQty", count("*").over(window_spec))
df_transactions = df_transactions.withColumn("RunSmallAmt", min("TranAmt").over(window_spec))
df_transactions = df_transactions.withColumn("RunLargeAmt", max("TranAmt").over(window_spec))
df_transactions = df_transactions.withColumn("RowNumber", row_number().over(window_spec)) #uzycie ROW_NUMBER 

In [0]:
#użycie funkcji okienkowych LEAD, LAG, FIRST_VALUE, LAST_VALUE
df_transactions = df_transactions.withColumn("LeadTranAmt", lead("TranAmt").over(window_spec))
df_transactions = df_transactions.withColumn("LagTranAmt", lag("TranAmt").over(window_spec))
df_transactions = df_transactions.withColumn("FirstTranAmt", first_value("TranAmt").over(window_spec))
df_transactions = df_transactions.withColumn("LastTranAmt", last_value("TranAmt").over(window_spec))


In [0]:
df_transactions = df_transactions.withColumn(
    "SlideAvg", avg("TranAmt").over(window_spec.rowsBetween(-2, 0))
)
df_transactions = df_transactions.withColumn(
    "SlideQty", count("*").over(window_spec.rowsBetween(-2, 0))
)
df_transactions = df_transactions.withColumn(
    "SlideMin", min("TranAmt").over(window_spec.rowsBetween(-2, 0))
)
df_transactions = df_transactions.withColumn(
    "SlideMax", max("TranAmt").over(window_spec.rowsBetween(-2, 0))
)
df_transactions = df_transactions.withColumn(
    "SlideTotal", sum("TranAmt").over(window_spec.rowsBetween(-2, 0))
)


In [0]:
df_transactions.show()

+---------+----------+-------+-----------+------------------+----------+-----------+-----------+---------+------------------+--------+--------+--------+----------+
|AccountId|  TranDate|TranAmt|RunTotalAmt|            RunAvg|RunTranQty|RunSmallAmt|RunLargeAmt|RowNumber|          SlideAvg|SlideQty|SlideMin|SlideMax|SlideTotal|
+---------+----------+-------+-----------+------------------+----------+-----------+-----------+---------+------------------+--------+--------+--------+----------+
|        1|2011-01-01|    500|        500|             500.0|         1|        500|        500|        1|             500.0|       1|     500|     500|       500|
|        1|2011-01-15|     50|        550|             275.0|         2|         50|        500|        2|             275.0|       2|      50|     500|       550|
|        1|2011-01-22|    250|        800| 266.6666666666667|         3|         50|        500|        3| 266.6666666666667|       3|      50|     500|       800|
|        1|2011-

In [0]:
df_logical = df_logical.withColumn(
    "SumByRows", sum("Salary").over(Window.orderBy("Salary").rowsBetween(Window.unboundedPreceding, 0))
)
df_logical = df_logical.withColumn(
    "SumByRange", sum("Salary").over(Window.orderBy("Salary").rangeBetween(Window.unboundedPreceding, 0))
)


In [0]:
df_logical.show()

+-----+-----------+------+---------+----------+
|RowID|      FName|Salary|SumByRows|SumByRange|
+-----+-----------+------+---------+----------+
|    1|     George|   800|      800|       800|
|    2|        Sam|   950|     1750|      1750|
|    3|      Diane|  1100|     2850|      2850|
|    4|   Nicholas|  1250|     4100|      5350|
|    5|     Samuel|  1250|     5350|      5350|
|    6|   Patricia|  1300|     6650|      6650|
|    7|      Brian|  1500|     8150|      8150|
|    8|     Thomas|  1600|     9750|      9750|
|    9|       Fran|  2450|    12200|     12200|
|   10|     Debbie|  2850|    15050|     15050|
|   11|       Mark|  2975|    18025|     18025|
|   12|      James|  3000|    21025|     24025|
|   13|    Cynthia|  3000|    24025|     24025|
|   14|Christopher|  5000|    29025|     29025|
+-----+-----------+------+---------+----------+

